<a href="https://colab.research.google.com/github/kirubanath/ML-DS_learnings/blob/main/Kaggle_comp1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U --no-cache-dir gdown 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Documentation: 

(I included all the steps I did and all my learnings in the documentation rather than showing everything in the code since it will just reduce the readability of the document)

**Loading the data:**
1. Surprisingly, loading the data to google colab was a mojor learning. This is because the data itself was very big and my ususla way of doing things failed.

**Preprocessing/EDA:**
1. I first perfromed the usual steps of seeing the distribution of each column (data.hist()). The data itself looked very well cleaned and there were no missing values/ outliers. 

2. As the second step I plotted the correlation matrix of the input columns as a heatmap to understand the relationships between the columns. There were few columns that had high correlations, especially since the columns form 9-19 were derivative of first eight columns. This gave me the idea of using just these 10 columns in my model. Infact this actually produced comparable accuracy to using all the features (accuracy just reduced by 0.5%) and even the training time was much faster. In a practical application I will use only these 10 features to train my model since its more efficient, but for the sake of the competition where every bit matters I ended up choosing the entire dataset.

3. For the next step I tried reducing the training data itself and rather than using the entire set I only used 10% of the dataset to train my models. Surprisingly this aslo gave comparable accuracy (accuracy reduced by around 0.6%) with much faster training time. Again in real world cases where efficiency is more important than half percent accuracy, I would use this technique rather than using the entire dataset.

4. Next I tried adding noise to the input data by randomly selecting 25% of the column data and adding the column mean to it, to make the model more generalized. But again the end accuracy was slightly lower than when I did not use this technique. So I dropped this as well.

5. I tried standardizing both the train and test data togather and because of the data leakage, I actually got a slightly better accuracy than the base model. But again for the final submission, I did not use this method since it is not the best practice when it comes to real world applications.

6. I also observed that the some columns are very skewed and for the case of linear models like logistic regression it is important to make then as normal before fitting the data. So i tried various transformations like log transform for the right skewed columns and even powertransforms like boxcox for the entire columns. But again I only ended up getting lesser accuracy(even though in theory it should work better. This might be becasue of my limitations and maybe I am missing something here.) In the end had to drop this idea as well.

7. As a final step I noticed that the columns had lot of outliers when I plottted the boxplots for each of the columns. Using the winsorize method(I tried both the Z-score and IQR method first but this one gave better results) I capped the outliers and tried again. It did give improvement to the linear models but still at the end since I ended up choosing the boosting classifiers that use decision trees, I did not use that for the final submisison since removing outliers won't make much difference for decision trees.


**Model Selection: Default parameters:**
1. SVM: I couldn't check with svm models because it was taking a long time and and my machine kept getting interrupted.

2. Decision Trees: The accuracy from decision trees were the lowest at around 78%.

3. Logistic Regression: Similar to decision trees, the accuracy was very low at 78% 

4. Random Forest: The model accuracy was around 79%. So with hyperparameter tuning it may be possible to imporve the model.

5. Adaboost: All the boosting classifiers performed better than other models. Same goes for Adaboost. It was at around 79%.

6. XGBoost: It was one of the two best performing models with accuracy at 80%. This was seelcted for hyperparamter tuning.

7. HistGradientBoost: Along with XGboost, it was one of the two best performing models and was selected for hyperparamter tuning.

**Model Selection: Hyperparameter tuning:**
1. Random Forest: I tried hyperparameter tuning with GridSearchCv but again like in the case of SVM it was taking too long. Then I tried with RandomSearchCV, I did get some results but the accuracy was not increasing much and lingered around 79%. So I abandoned it, but Its just an intuition that maybe with proper hyperparamter random forest would have performed better. But again due to time constraint had to abandon that route.

2. XGboost: Just like in the case of Random Forest, exhaustive search was not viable, so used RandomsearchCV with CV =5 and noted the best paramters among the many iterations that I have done. I used those best paramters that I personally could find for this final submission.

3. HistGradientBoost: Again did many iterations of RandomSearch and used the best parameters for the final submission. Infact my best model was this one with the tuned parameters. I used this model for the final output submission to the competition.


In [ ]:
#importing necessary libraries for the final submission:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import xgboost as xgb

from sklearn.preprocessing import StandardScaler

from sklearn.ensemble import HistGradientBoostingClassifier

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from sklearn.metrics import accuracy_score

import random
import gdown

In [ ]:
url = "https://drive.google.com/uc?id=1_FZIrNug8svcEK5QMDdV2MoHmcCny8kJ&export=download"
output = "train.csv"
gdown.download(url, output)

url = "https://drive.google.com/uc?id=1_FAidr3sAiSWx6mPS26HMpWjVaiSTOXq&export=download"
output = "test.csv"
gdown.download(url, output)

Downloading...
From: https://drive.google.com/uc?id=1_FZIrNug8svcEK5QMDdV2MoHmcCny8kJ&export=download
To: /content/train.csv
100%|██████████| 1.24G/1.24G [00:07<00:00, 161MB/s] 
Downloading...
From: https://drive.google.com/uc?id=1_FAidr3sAiSWx6mPS26HMpWjVaiSTOXq&export=download
To: /content/test.csv
100%|██████████| 524M/524M [00:04<00:00, 111MB/s] 


'test.csv'

In [ ]:
data_train  = pd.read_csv('/content/train.csv')

In [ ]:
data_test  = pd.read_csv('/content/test.csv')

In [ ]:
data_train.shape

(3500000, 20)

In [ ]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3500000 entries, 0 to 3499999
Data columns (total 20 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   Unnamed: 0                int64  
 1   lepton_1_pT               float64
 2   lepton_1_eta              float64
 3   lepton_1_phi              float64
 4   lepton_2_pT               float64
 5   lepton_2_eta              float64
 6   lepton_2_phi              float64
 7   missing_energy_magnitude  float64
 8   missing_energy_phi        float64
 9   MET_rel                   float64
 10  axial_MET                 float64
 11  M_R                       float64
 12  M_TR_2                    float64
 13  R                         float64
 14  MT2                       float64
 15  S_R                       float64
 16  M_Delta_R                 float64
 17  dPhi_r_b                  float64
 18  cos(theta_r1)             float64
 19  class                     float64
dtypes: float64(19), int64(1)

In [ ]:
#remove the unnamed 0 column
data_train.drop(columns = ['Unnamed: 0'], inplace = True)
data_test.drop(columns = ['Unnamed: 0'], inplace = True)

In [ ]:
#splitting the train data into input and output:
input = data_train.drop(columns = ['class'])
output = data_train['class']

cols = input.columns

Next step is standardizing the data. As mentioned above, if we standardize both the test and train dataset together, we are getting higher accuracy. But since that is not a good practice, I am not doing that in the final submission.

In [ ]:
#standardizing
scaler = StandardScaler()

input = scaler.fit_transform(input)
data_test = scaler.transform(data_test)

In [ ]:
input = pd.DataFrame(input)
input.columns = cols

data_test = pd.DataFrame(data_test)
data_test.columns = cols

Other than the standardization, any other EDA/modification is only resulting in accuracy going down! This maybe due to the fact that the data is already well cleaned. Even if we remove the columns that are highly correlated accuracy goes down. Also finally since the last 10 columns are derived features of the first 8 columns I even tried to use just the last 10 columns but the accuracy was slightly lower (just by 0.5%). In practical situation I would definitely just use the last 10 columns since it was giving comparable accuracy and the training time was much faster. But just for the sake of competition, Iam using everything now.

We will use the train data itself and split it into train and test to check the accuracy of the model.

In [ ]:
output.value_counts()

0.0    1899670
1.0    1600330
Name: class, dtype: int64

In [ ]:
X_train, X_test, Y_train , Y_test = train_test_split(input, output, random_state = 42, train_size = 0.7, stratify = output)

This is where we select the model and do hyperparameter tuning. As menioned above, I used randomsearchcv to tune the hyperparamters. Since the results are different everytime I am directly using the parameters that worked best here.

Also for the models that worked best, It was XGboost classifier and HistgradientboostClassifier compared to every other model. Among these two hist is giving slightly better accuracy score. So for the final submission I am using that.

Finally during the hyperparmeter tuning, the X_train and X_test from above were used and the CV= 5 was taken for crossvalidation.

In [ ]:
model1 = HistGradientBoostingClassifier(max_iter=400, learning_rate=0.1,random_state=42,min_samples_leaf=100,max_leaf_nodes=35,validation_fraction=0.2)

In [ ]:
model1.fit(X_train,Y_train)

HistGradientBoostingClassifier(max_iter=400, max_leaf_nodes=35,
                               min_samples_leaf=100, random_state=42,
                               validation_fraction=0.2)

In [ ]:
Y_pred = model1.predict(X_test)

In [ ]:
accuracy_score(Y_pred,Y_test)

0.8027914285714286

In [ ]:
model2 = xgb.XGBClassifier(objective= 'binary:logistic',subsample = 1.0, colsample_bytree =1.0,n_jobs=16, tree_method = 'hist')

In [ ]:
model2.fit(X_train,Y_train)

XGBClassifier(colsample_bytree=1.0, n_jobs=16, subsample=1.0,
              tree_method='hist')

In [ ]:
Y_pred = model2.predict(X_test)
accuracy_score(Y_pred,Y_test)

0.7971171428571429

As we can see above, among the best models I have trained, HistGradinetBoostClassifier is working the best. So will use that for the final submission on Kaggle.

In [ ]:
Y_out = model1.predict(data_test)

In [ ]:
#writing the output to output file:
output_data = pd.DataFrame(Y_out) 

In [ ]:
output_data.to_csv('/content/output.csv', header=['class'], index_label='Id')

The resulting file that was submitted to kaggle is in the output.csv file. And the final accuracy is around 80%


Throughout this journey I had few major learnings other than what I already knew, even though I couldnt successfully use them in this particular project:
1. Feature transformations : I had learned what tranformations to apply to what kind of features based on the distribution and QQ plot. How this will effect the accuracy of linear models like Logistic regression.
2. Outliers : How to look for outliers and how to deal with them. 
3. How to add noise to the data to make it more generalized
4. How to read and understand documentation (earlier I just used Youtube!)

Is there anything else we can do other than pure Hyperparameter tuning to get a better result? Throughout the journey I felt may times that this time something will change since I tried a new method or had a new insight but none of them produced any result, so I am just wondering.
